In [1]:
import numpy as np
import os
import shutil
import subprocess



In [2]:
total_count = 20


replica_count = 10

n_clusters = int(total_count / replica_count)

nclients = n_clusters

In [3]:
for folder in os.listdir('.'):
    if 'config' in folder and folder!='config' and 'zip' not in folder and '.' not in folder:
        shutil.rmtree(folder)

In [4]:
os.system("aws --region ap-south-1 ec2 describe-instances --filter Name=instance-type,Values=t2.small --query 'Reservations[*].Instances[*].[PrivateIpAddress]' --output text > all_internal_ips")

os.system("aws --region ap-south-1 ec2 describe-instances  --filter Name=instance-type,Values=t2.small --query 'Reservations[*].Instances[*].[PublicIpAddress]' --output text > all_external_ips")


# os.system("aws --region us-west-2 ec2 describe-instances --filter Name=instance-type,Values=c4.4xlarge --query 'Reservations[*].Instances[*].[PrivateIpAddress]' --output text >> all_internal_ips")

# os.system("aws --region us-west-2 ec2 describe-instances  --filter Name=instance-type,Values=c4.4xlarge --query 'Reservations[*].Instances[*].[PublicIpAddress]' --output text >> all_external_ips")



# os.system("aws --region us-west-2 ec2 describe-instances  --query 'Reservations[*].Instances[*].[PrivateIpAddress]' --output text > all_internal_ips")

# os.system("aws --region us-west-2 ec2 describe-instances --query 'Reservations[*].Instances[*].[PublicIpAddress]' --output text > all_external_ips")


0

In [5]:
with open('all_internal_ips') as f:
    ipList = f.read().splitlines()
ipList = [x for x in ipList if x!='None']

In [6]:
ipList, len(ipList)

(['172.31.47.69',
  '172.31.34.165',
  '172.31.42.105',
  '172.31.33.39',
  '172.31.35.131',
  '172.31.43.130',
  '172.31.37.210',
  '172.31.40.47',
  '172.31.38.153',
  '172.31.36.116',
  '172.31.44.10',
  '172.31.46.202',
  '172.31.38.109',
  '172.31.37.173',
  '172.31.40.125',
  '172.31.34.123',
  '172.31.41.254',
  '172.31.42.93',
  '172.31.47.25',
  '172.31.42.153',
  '172.31.39.123',
  '172.31.33.218'],
 22)

In [7]:
clusterIpMap = {}

for i in range(len(ipList)):
    clusterIpMap[i] = ipList[i]

    
for i in range(nclients):
    
    clusterIpMap[7001+i] = ipList[-(nclients-i)]




clusterPortMap1 = {}
clusterPortMap2 = {}

for i in range(len(ipList) - nclients):
    
    clusterPortMap1[i] = str(11000+i*10)
    
for i in range(len(ipList) - nclients):
    clusterPortMap2[i] = str(11000+i*10+1)
    
# clusterPortMap1[7003] = '11400'
# clusterPortMap2[7003] = '11410'



temp = 11900

for i in range(nclients):
    
    clusterPortMap1[7001+i] = str(temp+i*20)
    clusterPortMap2[7001+i] = str(temp+i*20+10)



In [8]:
clusterPortMap1, clusterPortMap2

({0: '11000',
  1: '11010',
  2: '11020',
  3: '11030',
  4: '11040',
  5: '11050',
  6: '11060',
  7: '11070',
  8: '11080',
  9: '11090',
  10: '11100',
  11: '11110',
  12: '11120',
  13: '11130',
  14: '11140',
  15: '11150',
  16: '11160',
  17: '11170',
  18: '11180',
  19: '11190',
  7001: '11900',
  7002: '11920'},
 {0: '11001',
  1: '11011',
  2: '11021',
  3: '11031',
  4: '11041',
  5: '11051',
  6: '11061',
  7: '11071',
  8: '11081',
  9: '11091',
  10: '11101',
  11: '11111',
  12: '11121',
  13: '11131',
  14: '11141',
  15: '11151',
  16: '11161',
  17: '11171',
  18: '11181',
  19: '11191',
  7001: '11910',
  7002: '11930'})

In [9]:
os.system('echo 3108 | sudo -S ./gradlew installDist')

[sudo] password for tejas: 


> Task :compileJava

> Task :processResources NO-SOURCE
> Task :classes
> Task :jar


Note: /home/tejas/work/BftSmart_Final/src/main/java/bftsmart/reconfiguration/ViewController.java uses or overrides a deprecated API.
Note: Recompile with -Xlint:deprecation for details.
Note: Some input files use unchecked or unsafe operations.
Note: Recompile with -Xlint:unchecked for details.


> Task :installDist

BUILD SUCCESSFUL in 6s
3 actionable tasks: 3 executed


0

In [ ]:
for i in range(total_count):
    os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(i)+' 1024')

    

os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(7002)+' 1024')
os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(7001)+' 1024')

In [ ]:
os.system('echo 3108 | sudo -S ./gradlew clean')

In [ ]:
for key in clusterPortMap1.keys():
    if key>7002:
        print(key)
        print()
        
        for folder in os.listdir('config/'):
            if os.path.isdir('config/'+folder) and 'keys' in folder:
                if os.path.exists('config/'+folder+'/'+'privatekey7001'):
                    shutil.copy('config/'+folder+'/'+'privatekey7001','config/'+folder+'/'+'privatekey'+str(key))
                    shutil.copy('config/'+folder+'/'+'publickey7001','config/'+folder+'/'+'publickey'+str(key))
                
        print('----------')
        print()

In [ ]:
for i in range(n_clusters):
    
    if os.path.exists('config'+str(i)):
        shutil.rmtree('config'+str(i))
    
    os.mkdir('config'+str(i))
    os.mkdir('config'+str(i)+'/keysRSA')
    os.mkdir('config'+str(i)+'/keysSunEC')
    os.mkdir('config'+str(i)+'/keysECDSA')
    
    
    shutil.copytree('config/workloads/', 'config'+str(i)+'/workloads')
    shutil.copytree('config/keysSSL_TLS/', 'config'+str(i)+'/keysSSL_TLS')
    
    
    for file in os.listdir('config/keys/'):
    
        shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysRSA/'+file)
        shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysSunEC/'+file)
        shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysECDSA/'+file)
    
    
    shutil.copy('config/system.config', 'config'+str(i)+'/system.config')    
    shutil.copy('config/logback.xml', 'config'+str(i)+'/logback.xml')
    
    
    
    

In [ ]:
        
        for i in range(nclients):
            print(str(7001+i)+' '+clusterIpMap[7001+i]+' '+clusterPortMap1[7001+i]+'\n')

In [ ]:
for n_cluster in range(n_clusters):
    
    with open('config'+str(n_cluster)+'/hosts.config','w') as file:
        
        for n_cluster1 in range(n_clusters):
        
            for i_replica in range(replica_count):
    #             file.writelines(str(i_replica)+' '+clusterIpMap[n_cluster*replica_count+i_replica]+' '  +clusterPortMap1[n_cluster*replica_count+i_replica]+' ' +  clusterPortMap2[n_cluster*replica_count+i_replica]+'\n')
                file.writelines(str(n_cluster1*replica_count+i_replica)+' '+clusterIpMap[n_cluster1*replica_count+i_replica]+' '  +clusterPortMap1[n_cluster1*replica_count+i_replica]+' ' +  clusterPortMap2[n_cluster1*replica_count+i_replica]+' '+str(n_cluster1)+'\n')

        for iter_ in range(len(ipList) - (n_clusters*replica_count)-nclients):
            print((n_clusters*replica_count) + iter_)
            
#             file.writelines(str((n_clusters*replica_count) + iter_)+' '+clusterIpMap[(n_clusters*replica_count) + iter_]+' '  +clusterPortMap1[(n_clusters*replica_count) + iter_]+' ' +  clusterPortMap2[(n_clusters*replica_count) + iter_]+' '+str(iter_%n_clusters)+'\n')
            file.writelines(str((n_clusters*replica_count) + iter_)+' '+clusterIpMap[(n_clusters*replica_count) + iter_]+' '  +clusterPortMap1[(n_clusters*replica_count) + iter_]+' ' +  clusterPortMap2[(n_clusters*replica_count) + iter_]+' '+str(0%n_clusters)+'\n')
        
        for i in range(nclients):
            file.writelines(str(7001+i)+' '+clusterIpMap[7001+i]+' '+clusterPortMap1[7001+i]+'\n')

        
        file.close()


In [ ]:
replica_count

In [ ]:
for n_cluster in range(n_clusters):
    with open('config'+str(n_cluster)+'/system.config','r') as file:
        data = file.readlines()
        f = int(np.ceil((replica_count-1)/3))
        print('f is',f )
        
        for n_line in range(len(data)):
            if ('system.servers.num' in data[n_line]) and '#' not in data[n_line]:
                
                data[n_line] = 'system.servers.num = '+str(replica_count)
                
            if ('system.servers.f' in data[n_line]) and '#' not in data[n_line]:
                data[n_line] = 'system.servers.f = '+str(f)
                
            if ('system.initial.view' in data[n_line]) and '#' not in data[n_line]:
                data[n_line] = 'system.initial.view = '
                for iter_view in range(replica_count):
                    data[n_line] = data[n_line] +str(n_cluster*replica_count+iter_view)+','
#                     data[n_line] = data[n_line] +str(n_cluster*replica_count+iter_view)+','
                
                data[n_line] = data[n_line][:-1]
                
                
        file.close()
        
    with open('config'+str(n_cluster)+'/system.config', 'w') as f:
        for line in data:
            f.write(f"{line}\n")
        

In [ ]:
for n_cluster in range(n_clusters):
    for i in range(400):
        
        for ncls in  range(1):
        
        
            print('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
                        'config'+str(n_cluster)+'/keysRSA/'+'publickey'+str(1000+(ncls*100)+i))
            shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
                        'config'+str(n_cluster)+'/keysRSA/'+'publickey'+str(1000+(ncls*100)+i))
            shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'privatekey7002', \
                        'config'+str(n_cluster)+'/keysRSA/'+'privatekey'+str(1000+(ncls*100)+i))
        
#         shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
#                     'config'+str(n_cluster)+'/keysRSA/'+'publickey7002')
        
#         shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'privatekey7002', \
#                     'config'+str(n_cluster)+'/keysRSA/'+'privatekey7002')

In [ ]:
# os.system('git remote set-url origin https://github.com/tmane002/BftSmart_Final.git')

In [ ]:
os.system('git add .; git commit -m "Working with modified ClusterInfo";git push')

In [ ]:
# 'gcloud compute instances create instance-1 --project=cellular-hybrid-358402 --zone=us-central1-a --machine-type=e2-medium --network-interface=network-tier=PREMIUM,subnet=default --no-restart-on-failure --maintenance-policy=TERMINATE --provisioning-model=SPOT --instance-termination-action=STOP --service-account=932709639592-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-1,image=projects/ubuntu-os-cloud/global/images/ubuntu-2204-jammy-v20220810,mode=rw,size=10,type=projects/cellular-hybrid-358402/zones/us-central1-a/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --reservation-affinity=any'

In [ ]:
# gcloud compute instances create instance-1 --project=cellular-hybrid-358402 --zone=us-west1-b --machine-type=e2-medium --network-interface=network-tier=PREMIUM,subnet=default --no-restart-on-failure --maintenance-policy=TERMINATE --provisioning-model=SPOT --instance-termination-action=STOP --service-account=932709639592-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-1,image=projects/ubuntu-os-cloud/global/images/ubuntu-2204-jammy-v20220810,mode=rw,size=10,type=projects/cellular-hybrid-358402/zones/us-central1-a/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --reservation-affinity=any